In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import BayesianRidge, LinearRegression


In [2]:
# Load Data
data = pd.read_csv('Key_indicator_districtwise.csv')
features = data.columns
y_labels = ['YY_Infant_Mortality_Rate_Imr_Total_Person']
y = data[y_labels].values
x_raw = data.values
x_raw2 = np.delete(x_raw,590,1)
x_raw_labels = features.delete(590)
x1 = x_raw2.copy()
x_raw_labels1 = x_raw_labels.copy()

for i in range(17,0,-3):
    x1[:, i+1] = x1[:, i+1]/x1[:, i]
    x1[:, i+2] = x1[:, i+2]/x1[:, i]
    x1 = np.delete(x1, obj=i, axis=1)
    x_raw_labels1 = np.delete(x_raw_labels1, obj=1)

In [10]:
# make state and district categorical variables
states = pd.get_dummies(x_raw[:,0])
dists = pd.get_dummies(x_raw[:,1])
state_cols = states.columns
dists_cols = dists.columns
new_cols = np.append(state_cols, dists_cols)
new_vars = np.append(states, dists, axis=1)
# then append
x1 = np.delete(x1, obj=1, axis=1)
x1 = np.delete(x1, obj=0, axis=1)
x_raw_labels1 = np.delete(x_raw_labels1, obj=1)
x_raw_labels1 = np.delete(x_raw_labels1, obj=0)

#x_raw_labels1 = np.append(new_cols, x_raw_labels1)
#x_new = np.append(new_vars, x1, axis=1)
#x = x_new
#x_labels = x_raw_labels1
x = x1
x_labels = x_raw_labels1

In [12]:
# Remove NaNs (impute)
z = np.zeros(x.shape[1])
for i in range(0,x.shape[0]):
    for j in range(0,x.shape[1]):
        z[j] += np.isnan(x[i,j])

In [13]:
print(z.shape)
print(x.shape)
print(x_labels.shape)
x_no_nans = x[:,z == 0]
x_no_nans_labels = x_labels[z == 0]

(633,)
(284, 633)
(633,)


In [14]:
clf = BayesianRidge(compute_score=True, normalize=True)
clf.fit(x_no_nans, y)

ols = LinearRegression(normalize=True)
ols.fit(x_no_nans, y)

//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [15]:
bayesian_coefs = clf.coef_

In [16]:
sorted_coefs = np.sort(np.abs(bayesian_coefs))
sorted_indices = np.argsort(np.abs(bayesian_coefs))
sorted_labels = x_no_nans_labels[sorted_indices]

In [ ]:
sorted_labels[-100:]